In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs               
import cartopy.feature as cfeature         
import cartopy.util as cutil
import xarray as xr                        
import numpy as np
import pandas as pd
import plotting_module
import xesmf as xe

In [2]:
ozone_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.O3.200201-202412.nc")
pdeldry_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PDELDRY.200201-202412.nc")
ps_dataset = xr.open_dataset('/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PS.200201-202412.nc')

In [3]:
ozone = ozone_dataset["O3"]

In [4]:
p0 = ozone_dataset["P0"]
hyai = ozone_dataset["hyai"]
hybi = ozone_dataset["hybi"]
ps = ps_dataset['PS']
pdeldry = pdeldry_dataset['PDELDRY']
lev = ozone_dataset.coords['lev']
num_lev = lev.shape[0]

# convert to hPa from Pa
p0 = p0.copy() / 100
ps = ps.copy() / 100
pdeldry = pdeldry.copy() / 100 

# truncate to levels 16-31
truncated_pdeldry = pdeldry.isel({pdeldry.dims[1]: slice(17, 31)})
truncated_ozone = ozone.isel({ozone.dims[1]: slice(17, 31)})

Now group 'ozone' and 'pdeldry' DataArrays by months

In [5]:
start_date = '2005-02-01'
end_date = '2025-01-01'

# group the 240 month dates based on calendar months for both PDELDRY and O3 variables

truncated_pdeldry = truncated_pdeldry.sel(time=slice(start_date, end_date))
pdeldry_monthly_mean = truncated_pdeldry.groupby('time.month').mean('time')
pdeldry_monthly_mean = pdeldry_monthly_mean.transpose('lev','month','lat','lon')

truncated_ozone = truncated_ozone.sel(time=slice(start_date, end_date))
ozone_monthly_mean = truncated_ozone.groupby('time.month').mean('time')
ozone_monthly_mean = ozone_monthly_mean.transpose('lev','month','lat','lon')

In [6]:
# constants / conversion factor
NAv = 6.0221415e+23                       # molecules in mole
g = 9.81                                  # gravity
MWair = 28.94                             # g/mol
xp_const = (NAv * 10)/(MWair*g)           # scaling factor, pa to hPa and cm to m
DU_CONVERSION = 2.69 * 10**16

In [ ]:
# Initialize pressure edge arrays
mod_press_low = xr.zeros_like(ozone).transpose('lev','lat','lon','time')
mod_press_top = xr.zeros_like(ozone).transpose('lev','lat','lon','time')

Calculating pressure at hybrid levels

p(k) = a(k) * p0 + b(k) * ps

In [ ]:
# Calculate pressure edge arrays
# CAM-chem layer indices start at the top and end at the bottom
for i in range(num_lev):
    mod_press_top[i,:,:,:] = hyai[i]*p0 + hybi[i]*ps

In [ ]:
mod_press_top = mod_press_top.transpose('lev', 'time', 'lat', 'lon')

In [ ]:
filtered_300hpa_upper = mod_press_top.where(mod_press_top >= 300, drop=False)
filtered_300hpa_upper = filtered_300hpa_upper.where(mod_press_top < 322.24, drop=False)

In [ ]:
mod_deltap = abs(300 - filtered_300hpa_upper)

In [ ]:
mod_deltap[17][36][10]

In [ ]:
ozone_filtered = ozone.where(mod_deltap.notnull())
ozone_filtered = ozone_filtered.transpose('lev','time','lat','lon')

In [ ]:
mod_deltap_filtered = mod_deltap.where(ozone_filtered.notnull())

In [ ]:
mod_deltap_filtered = mod_deltap_filtered.fillna(0)
ozone_filtered = ozone_filtered.fillna(0)

In [ ]:
mod_deltap_filtered[17][36][10]

In [ ]:
mod_deltap_filtered = mod_deltap_filtered.sel(time=slice(start_date, end_date))
ozone_filtered = ozone_filtered.sel(time=slice(start_date, end_date))

mod_deltap_filtered = mod_deltap_filtered.groupby('time.month').mean('time')
ozone_filtered = ozone_filtered.groupby('time.month').mean('time')

In [ ]:
mod_deltap_filtered.mean(dim={'lev', 'month','lat','lon'}, skipna=True)

Calculating ozone column for [300hPa, 322.24hPa) and averaging it

In [ ]:
ozone_300hpa_column = xr.dot(mod_deltap_filtered, xp_const*ozone_filtered, dims='lev')

In [ ]:
ozone_du_300hpa_column = ozone_300hpa_column.copy() / DU_CONVERSION

ozone_du_300hpa_column = ozone_du_300hpa_column.where(ozone_du_300hpa_column != 0)

In [ ]:
#ozone_du_300hpa_column = ozone_du_300hpa_column.sel(time=slice(start_date, end_date))
#ozone_du_300hpa_column = ozone_du_300hpa_column.groupby('time.month').mean('time')

Now calculating ozone column from 322.24hPa to ground level

In [ ]:
ozone_part_column = xr.dot(pdeldry_monthly_mean, xp_const*ozone_monthly_mean, dims='lev')

In [ ]:
ozone_du_column = ozone_part_column.copy() / DU_CONVERSION

Now calculate total tropospheric column ozone

In [ ]:
ozone_du_300hpa_column = ozone_du_300hpa_column.fillna(0)
total_tco = ozone_du_column + ozone_du_300hpa_column

In [ ]:
total_tco

Regridding and plotting

In [ ]:
omi_mls_ds = xr.open_dataarray("/glade/u/home/mvoncyga/SOARS_2025/OMIMLS_300hpa_monthly_mean_2005_2024.nc")
omi_mls_ds = omi_mls_ds.rename({'latitude': 'lat', 'longitude': 'lon'})

# shifting lon to be 0-360
omi_mls_ds['lon'] = omi_mls_ds['lon'] % 360
omi_mls_ds = omi_mls_ds.sortby('lon')

In [ ]:
#ozone_du_column_regridded = ozone_du_column.interp(coords=dict(lat=omi_mls_ds.lat, lon=omi_mls_ds.lon), method='linear')
regridder = xe.Regridder(total_tco, omi_mls_ds, 'bilinear', periodic=True)                                                                             

In [ ]:
cesm_regridded = regridder(total_tco)

In [ ]:
cesm_regridded

In [ ]:
month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for i in range(12):
    plt.figure(figsize=(10,8))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # plotting data
    cesm_regridded[i].plot(vmin=5, vmax=40)
    
    ax.set_title("CESM $\mathregular{O_3}$ Concentration, "+ str(month_list[i]), fontsize=18)
    
    # add coastlines
    ax.add_feature(cfeature.COASTLINE, linestyle='--')

    # add lat lon grids
    gl = ax.gridlines(draw_labels=True, color='grey', alpha=0.8, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    
    plt.show()

In [ ]:
difference = cesm_regridded - omi_mls_ds

In [ ]:
difference.mean(dim={'month','lat','lon'}, skipna=True)

In [ ]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l'] # for cycling through figures to create a gif

i = 0
for i in range(12):
    
    plt.figure(figsize=(10, 6))
    ax.set_facecolor('gray')
    ax = plt.axes(projection=ccrs.PlateCarree())
    difference[i].plot(ax=ax,transform=ccrs.PlateCarree(central_longitude=0), vmin = -20, vmax = 20, extend='both', cmap='bwr')
    ax.add_feature(cfeature.COASTLINE, linestyle='--')
    ax.set_global()

    # adding gridlines
    ax.gridlines(draw_labels=True, color='black', alpha=0.5, linestyle='--')
    ax.set_title("CESM - OMI/MLS Tropo O3 Difference: " + month_list[i] + ", 2005-2024", fontsize=14)
    
    formatted_mean = f"{difference[i].mean().data:.3f}"

    ax.text(0.10, -0.25, 'Mean: ' + str(formatted_mean) + ' DU', va='bottom', ha='center',
            rotation='horizontal', rotation_mode='anchor',
            transform=ax.transAxes, fontsize=12)
    

    #plt.savefig("figures/CESM_Diff-" + str(alphabet[i]))
    i+=1

    plt.show

In [ ]:
import imageio
import os

def create_gif(image_folder, output_gif, duration):
    filenames = sorted([f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))])
    images = []
    for filename in filenames:
        if filename.startswith("CESM_Diff"):
            image_path = os.path.join(image_folder, filename)
            images.append(imageio.imread(image_path))
    imageio.mimsave(output_gif, images, duration=duration)

image_dir = '/glade/u/home/mvoncyga/SOARS_2025/figures/'

#create_gif(image_dir, "Difference_Plot.gif", duration=1000)